In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display,Javascript,HTML
from Helperfunctions import clean_df_11pick
from PIL import Image, ImageFont, ImageDraw, ImageOps
import locale
import os
import io
import base64

locale.setlocale(locale.LC_ALL, '')  # Set the locale to the user's default

lineup11 = widgets.Output()
filtering = widgets.Output()
alert = widgets.Output()


def start_11_lineup(_):
    # Remove the budget input widget and start the lineup selection
    with output_box:
        display_button.close()
        budget_input.close()
        display(widgets.HTML('<h5 style="background-color: #77ffe4; font-size: 14px; font-weight: bold; padding: 8px;">' +'Select your 11-player lineup:'+ '</h5>', layout=widgets.Layout(width='200px')))
        display(filter_widget)
        # Add the output widget
        display(output)
        display(team_output)
        display(lineup11)

# Step 2: Load the DataFrame with soccer player data
df = clean_df_11pick()

# Step 3: Create interactive widgets for user input
budget_input = widgets.IntText(description='Budget:')
# Create a button widget for starting the lineup selection
display_button = widgets.Button(description='Start my 11-player lineup', layout=widgets.Layout(width='200px'),button_style='primary')
display_button.on_click(start_11_lineup)

league_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">League:</span>')
continent_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Continent:</span>')
club_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Club:</span>')
nationality_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Nationality:</span>')
position_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Position:</span>')
overall_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Overall:</span>')
potential_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Potential:</span>')
age_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Age:</span>')
market_value_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Market Value:</span>')
sort_dropdown_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Sort By:</span>')
order_dropdown_description = widgets.HTML(value='<span style="background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;">Order By:</span>')

# Create the dropdown widgets
league_dropdown = widgets.Dropdown(options=['Any', 'Top 5 Leagues','Not in Top 5 League'] + sorted(df['league_name'].unique()),layout=widgets.Layout(width='300px'))
combined_league_widget = widgets.HBox([league_description, league_dropdown])

continent_dropdown = widgets.Dropdown(options=['Any', 'Africa', 'Asia', 'Europe', 'North America', 'South America', 'Oceania'],layout=widgets.Layout(width='300px'))
combined_continent_widget = widgets.HBox([continent_description, continent_dropdown])

club_dropdown = widgets.Dropdown(options=['Any'], layout=widgets.Layout(width='300px'))
combined_club_widget = widgets.HBox([club_description, club_dropdown])

nationality_dropdown = widgets.Dropdown(options=['Any'], layout=widgets.Layout(width='300px'))
combined_nationality_widget = widgets.HBox([nationality_description, nationality_dropdown])


position_dropdown = widgets.Dropdown(options=['Any'] + sorted(df['best_position'].unique()), layout=widgets.Layout(width='300px'))
combined_position_widget = widgets.HBox([position_description, position_dropdown])


overall_input = widgets.IntText(value=0, layout=widgets.Layout(width='300px'))
overall_option_button = widgets.ToggleButtons(value='>=',options=['<=', '>=', '='])
combined_overall_widget = widgets.HBox([overall_description,overall_input, overall_option_button])


potential_input = widgets.IntText(value=0, layout=widgets.Layout(width='300px'))
potential_option_button = widgets.ToggleButtons(value='>=',options=['<=', '>=', '='])
combined_potential_widget = widgets.HBox([potential_description,potential_input, potential_option_button])

age_input = widgets.IntText(value=0, layout=widgets.Layout(width='300px'))
age_option_button = widgets.ToggleButtons(value='>=',options=['<=', '>=', '='])
combined_age_widget = widgets.HBox([age_description,age_input, age_option_button])

market_value_input = widgets.IntText(value=0, description='Market Value:')
market_value_option_button = widgets.ToggleButtons(value='>=',options=['<=', '>=', '='])
combined_market_value_widget = widgets.HBox([market_value_description,market_value_input, market_value_option_button])

sort_dropdown = widgets.Dropdown(options=['Overall', 'Potential', 'Age', 'Market Value', 'Pace', 'Shooting', 'Passing', 'Dribling', 'Defending', 'Physical', 'Diving', 'Handling', 'Kicking', 'Reflexes', 'Speed', 'Positioning'], layout=widgets.Layout(width='300px'))
sort_order_dropdown = widgets.Dropdown(options=['Descending','Ascending'], layout=widgets.Layout(width='300px'))
combined_sort_widget =widgets.HBox([sort_dropdown_description,sort_dropdown,order_dropdown_description, sort_order_dropdown])



filter_button = widgets.Button(description='Filter',button_style='info')

# Create the filter widget
filter_widget = widgets.VBox([
    combined_league_widget,
    combined_club_widget,
    combined_continent_widget,
    combined_nationality_widget,
    combined_position_widget,
    combined_overall_widget,
    combined_potential_widget,
    combined_age_widget,
    combined_market_value_widget,
    combined_sort_widget,
    filter_button,
    filtering
])


# Output widget for displaying the filtered results
output = widgets.Output()

# Define a dictionary mapping continents to nationalities
countries_dict = {
    'Any': df['nationality'].unique().tolist(),
    'Africa': df[df['continent'] == 'Africa']['nationality'].unique().tolist(),
    'Asia': df[df['continent'] == 'Asia']['nationality'].unique().tolist(),
    'Europe': df[df['continent'] == 'Europe']['nationality'].unique().tolist(),
    'North America': df[df['continent'] == 'North America']['nationality'].unique().tolist(),
    'South America': df[df['continent'] == 'South America']['nationality'].unique().tolist(),
    'Oceania': df[df['continent'] == 'Oceania']['nationality'].unique().tolist()
}

def update_clubs(*args):
    selected_league = league_dropdown.value
    if selected_league == 'Top 5 Leagues':
        clubs = ['Any'] + sorted(df[df['league_name'].isin(['[Spain 1st Tier] LaLiga',
                                                            '[England 1st Tier] Premier League',
                                                            '[France 1st Tier] Ligue 1',
                                                            '[Germany 1st Tier] Bundesliga',
                                                            '[Italy 1st Tier] Serie A'])]['club_name'].unique())

    elif selected_league =='Not in Top 5 League':
        clubs = ['Any'] + sorted(df[~df['league_name'].isin(['[Spain 1st Tier] LaLiga',
                                                            '[England 1st Tier] Premier League',
                                                            '[France 1st Tier] Ligue 1',
                                                            '[Germany 1st Tier] Bundesliga',
                                                            '[Italy 1st Tier] Serie A'])]['club_name'].unique())

    else:
        clubs = ['Any'] + sorted(df[df['league_name'] == selected_league]['club_name'].unique())
    club_dropdown.options = clubs



# Function to update nationality options based on the selected continent
def update_nationalities(*args):
    selected_continent = continent_dropdown.value
    nationalities = countries_dict[selected_continent]
    nationality_dropdown.options = ['Any'] + sorted(nationalities)
    

# Step 4: Create a function to handle filtering, sorting, and pagination of the DataFrame based on user input
current_page = [0]  # Current page number

# Create a list to store selected players
team_players = []
def add_to_team(player):
    def add_button_clicked(_):
        alert.clear_output()
        with alert:
            if len(team_players) >= 11:
                display(Javascript('alert("Team is already full with 11 players.")'))
                return
            elif player.name in team_players:
                display(Javascript('alert("Player is already in the team.")'))
                return
            elif budget_input.value >= player['market_value']:
                team_players.append(player.name)
                budget_input.value -= player['market_value']
                update_team_list()
            else:
                display(Javascript('alert("Not enough budget to add player to the team.")'))
                return

    add_button = widgets.Button(description='Add to my team', button_style='success', layout=widgets.Layout(width='150px', height='50px'))
    add_button.on_click(add_button_clicked)

    return add_button


def remove_from_team(player_index):
    player = df.loc[player_index]  # Retrieve the player from the DataFrame using the index
    team_players.remove(player_index)  # Remove the player index from the team_players list
    budget_input.value += player['market_value']
    update_team_list()
    if(len(team_players)<11):
        lineup11.clear_output()
    

def show_starting_11(df, selected_positions):
    lineup11.clear_output()
    with lineup11:
        background_image_path = 'Possible Positions.png'
        positions_coordinates = {
            'LS': (220, 90),'ST': (400, 90),'RS': (580, 90),
            'LW': (45, 280), 'LF': (220, 280), 'CF': (395, 280), 'RF': (575, 280), 'RW': (755, 280),
            'LAM': (220, 460), 'CAM': (400, 460), 'RAM': (580, 460),
            'LM': (45, 640), 'LCM': (220, 640), 'CM': (400, 640), 'RCM': (580, 640), 'RM': (755, 640),
            'LWB': (45, 820), 'LDM': (220, 820), 'CDM': (400, 820), 'RDM': (580, 820), 'RWB': (755, 820),
            'LB': (45, 1000), 'LCB': (220, 1000), 'CB': (400, 1000), 'RCB': (580, 1000), 'RB': (755, 1000),
            'GK': (400, 1180)
        }

        # Load the background image
        background_image = Image.open(background_image_path).convert("RGBA")

        # Create a new image with transparency
        composed_image = Image.new('RGBA', background_image.size, (0, 0, 0, 0))

        # Paste the background image onto the composed image
        composed_image.paste(background_image, (0, 0))
        total_market_value =0
        # Iterate over the selected positions and paste the player images
        for player_index, position in selected_positions:
            player_row = df.loc[player_index]
            player_name = player_row['name']
            player_sofifa_id = player_row['sofifa_id']
            player_overall = player_row['overall']
            position_overall = player_row[position]
            player_nationality = player_row['nationality']
            player_club = player_row['club_name']
            player_position = player_row['best_position']
            total_market_value += player_row['market_value']
            # Create the name to be displayed
            if len(player_name.split()) > 1:
                first_name, *rest_of_names = player_name.split()
                displayed_name = f"{first_name[0]}. {' '.join(rest_of_names)}"

            else:
                displayed_name = player_name

            # Get the coordinates for the player's position
            position_coordinates = positions_coordinates[position]

            # Calculate the coordinates to paste the player image
            x, y = position_coordinates
            paste_x = x - 40
            paste_y = y - 40

            # Load player image based on overall rating and position
            if player_overall >= 75:
                if player_position == 'GK':
                    player_image = Image.open('Fifa cards/Gold GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Gold.png').convert("RGBA")
            elif player_overall >= 65:
                if player_position == 'GK':
                    player_image = Image.open('Fifa cards/Silver GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Silver.png').convert("RGBA")
            else:
                if player_position == 'GK':
                    player_image = Image.open('Fifa cards/Bronze GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Bronze.png').convert("RGBA")


            # Resize the player image
            player_image = player_image.resize((200, 200))

            # Paste the player image onto the composed image
            composed_image.alpha_composite(player_image, (paste_x, paste_y))

            # Add the name below the player image
            name_x = paste_x + player_image.size[0] // 2 - len(displayed_name) * 4
            name_y = paste_y + player_image.size[1] - 90
            name_font = ImageFont.truetype("times.ttf", 18)
            draw = ImageDraw.Draw(composed_image)
            draw.text((name_x, name_y), displayed_name, fill=(0, 0, 0), font=name_font)

            # Load the nationality flag image
            flag_image_path = f"Countries/{player_nationality}.png"
            if os.path.exists(flag_image_path):
                flag_image = Image.open(flag_image_path).convert("RGBA")
            else:
                flag_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

            # Resize the flag image
            flag_image = flag_image.resize((23, 15))

            # Calculate the coordinates to paste the flag image
            flag_x = paste_x - flag_image.size[0] + 68
            flag_y = paste_y + (player_image.size[1] - flag_image.size[1]) // 2 - 26

            # Paste the flag image on the composed image
            composed_image.alpha_composite(flag_image, (flag_x, flag_y))

            # Load the team logo image
            team_logo_path = f"Club/{player_club}.png"
            if os.path.exists(team_logo_path):
                team_logo = Image.open(team_logo_path).convert("RGBA")
            else:
                team_logo = Image.new("RGBA", (1, 1))  # Use a blank image if team logo doesn't exist

            # Resize the team logo image
            team_logo = team_logo.resize((22, 22))

            # Calculate the coordinates to paste the team logo image
            logo_x = flag_x
            logo_y = flag_y + flag_image.size[1] + 7

            # Paste the team logo image on the composed image
            composed_image.alpha_composite(team_logo, (logo_x, logo_y))

            # Load the Sofifa profile image
            profile_image_path = f"Players/{player_sofifa_id}.png"
            if os.path.exists(profile_image_path):
                player_image_sofifa = Image.open(profile_image_path).convert("RGBA")
            else:
                player_image_sofifa = Image.new("RGBA", (1, 1))  # Use a blank image if profile image doesn't exist

            # Resize the Sofifa image
            player_image_sofifa = player_image_sofifa.resize((90, 90))
            paste_x = x - player_image_sofifa.size[0] // 2 + 80
            paste_y = y - player_image_sofifa.size[1] // 2 + 25
            # Paste the Sofifa image on top of the player image
            composed_image.alpha_composite(player_image_sofifa, (paste_x, paste_y))


            # Add the best position above the flag
            best_position_x = flag_x + flag_image.size[0] // 2 - len(df.loc[player_index]['best_position']) * 5
            best_position_y = flag_y - 20
            best_position_font = ImageFont.truetype("cour.ttf", 18)
            draw.text((best_position_x, best_position_y), df.loc[player_index]['best_position'], fill=(0, 0, 0), font=best_position_font)

            # Add the overall rating above the best position
            overall_x = best_position_x -2
            overall_y = best_position_y - 25
            overall_font = ImageFont.truetype("timesbd.ttf", 25)
            draw.text((overall_x, overall_y), str(position_overall), fill=(0, 0, 0), font=overall_font)

                    # Add the pace attribute below the name
            pace_x = paste_x + player_image.size[0] // 2 - len(str(df.loc[player_index]['pace']))-125
            pace_y = paste_y + player_image.size[1]-87
            pace_font = ImageFont.truetype("times.ttf", 11)
            draw.text((pace_x, pace_y), str(df.loc[player_index]['pace']), fill=(0, 0, 0), font=pace_font)

            # Add the shooting attribute below pace
            shooting_x = pace_x
            shooting_y = pace_y + 15
            shooting_font = ImageFont.truetype("times.ttf", 11)
            draw.text((shooting_x, shooting_y), str(df.loc[player_index]['shooting']), fill=(0, 0, 0), font=shooting_font)

            # Add the passing attribute below shooting
            passing_x = pace_x
            passing_y = shooting_y + 15
            passing_font = ImageFont.truetype("times.ttf", 11)
            draw.text((passing_x, passing_y), str(df.loc[player_index]['passing']), fill=(0, 0, 0), font=passing_font)

            # Add the dribbling overall attribute to the right of pace
            dribbling_x = pace_x + 70
            dribbling_y = pace_y
            dribbling_font = ImageFont.truetype("times.ttf", 11)
            draw.text((dribbling_x, dribbling_y), str(df.loc[player_index]['dribbling_overall']), fill=(0, 0, 0), font=dribbling_font)

            # Add the defense attribute below dribbling overall
            defense_x = dribbling_x
            defense_y = dribbling_y + 15
            defense_font = ImageFont.truetype("times.ttf", 11)
            draw.text((defense_x, defense_y), str(df.loc[player_index]['defense']), fill=(0, 0, 0), font=defense_font)

            # Add the physical attribute below defense
            physical_x = dribbling_x
            physical_y = defense_y + 15
            physical_font = ImageFont.truetype("times.ttf", 11)
            draw.text((physical_x, physical_y), str(df.loc[player_index]['physical']), fill=(0, 0, 0), font=physical_font)

        fromated_total_market_value = locale.format_string("%d", total_market_value, grouping=True)
        # Add the total market value to the corner
        market_value_text = f"Total Market Value:\n € {fromated_total_market_value}"
        market_value_font = ImageFont.truetype("timesbd.ttf", 18)
        draw.text((10, 10), market_value_text, fill=(255, 255, 255), font=market_value_font)
                
        # Create a button for saving the image
        save_button = widgets.Button(description='Save Image')
        filename_text = widgets.Text(description='Filename:')
        save_output = widgets.Output()

        def save_image(button):
            save_output.clear_output()
            with save_output:
                filename = filename_text.value
                if not filename:
                    filename = "My-Starting-11"

                # Save the composed image temporarily on the server
                save_path = f'{filename}.png'
                composed_image.save(save_path)

                display(Javascript(f"alert('Image saved as {save_path}')"))

                # Generate download link
                with open(save_path, 'rb') as f:
                    data = f.read()
                b64 = base64.b64encode(data).decode('utf-8')

                # Remove the temporary image file
                os.remove(save_path)

                href = f'''
                <a id="download_link" href="data:image/png;base64,{b64}" download="{filename}.png" style="display:none;"></a>
                <script>
                    document.getElementById('download_link').click();
                </script>
                '''
                display(HTML(href))

        save_button = widgets.Button(description='Save Image', button_style='success')
        filename_text = widgets.Text(placeholder='Enter file name', description='File Name:')
        save_button.on_click(save_image)

        display(composed_image)

        # Display the save button and text input
        display(widgets.HBox([save_button, filename_text, save_output]))


def create_composed_image_widget(df, player_index):
    paste_x = 0
    paste_y = 0
    player = df.loc[player_index]

    if player['overall'] >= 75:
        if player['best_position'] == 'GK':
            player_image = Image.open('Fifa cards/Gold GK.png').convert("RGBA")
        else:
            player_image = Image.open('Fifa cards/Gold.png').convert("RGBA")
    elif player['overall'] >= 65:
        if player['best_position'] == 'GK':
            player_image = Image.open('Fifa cards/Silver GK.png').convert("RGBA")
        else:
            player_image = Image.open('Fifa cards/Silver.png').convert("RGBA")
    else:
        if player['best_position'] == 'GK':
            player_image = Image.open('Fifa cards/Bronze GK.png').convert("RGBA")
        else:
            player_image = Image.open('Fifa cards/Bronze.png').convert("RGBA")

    player_image = player_image.resize((200, 200))
    transparent_background = Image.new("RGBA", player_image.size, (0, 0, 0, 0))
    composed_image = Image.alpha_composite(transparent_background, player_image)

    if len(player['name'].split()) > 1:
        first_name, *rest_of_names = player['name'].split()
        displayed_name = f"{first_name[0]}. {' '.join(rest_of_names)}"
    else:
        displayed_name = player['name']

    name_x = paste_x + player_image.size[0] // 2 - len(displayed_name) * 4
    name_y = paste_y + player_image.size[1] - 90
    name_font = ImageFont.truetype("times.ttf", 18)
    draw = ImageDraw.Draw(composed_image)
    draw.text((name_x, name_y), displayed_name, fill=(0, 0, 0), font=name_font)

    player_nationality = player['nationality']
    flag_image_path = f"Countries/{player_nationality}.png"
    if os.path.exists(flag_image_path):
        flag_image = Image.open(flag_image_path).convert("RGBA")
    else:
        flag_image = Image.new("RGBA", (1, 1))

    flag_image = flag_image.resize((23, 15))
    flag_x = paste_x - flag_image.size[0] + 68
    flag_y =  paste_y + (player_image.size[1] - flag_image.size[1]) // 2 - 26
    composed_image.alpha_composite(flag_image, (flag_x, flag_y))

    player_club = player['club_name']
    team_logo_path = f"Club/{player_club}.png"
    if os.path.exists(team_logo_path):
        team_logo = Image.open(team_logo_path).convert("RGBA")
    else:
        team_logo = Image.new("RGBA", (1, 1))

    team_logo = team_logo.resize((22, 22))
    logo_x = flag_x
    logo_y = flag_y + flag_image.size[1] + 7
    composed_image.alpha_composite(team_logo, (logo_x, logo_y))

    player_sofifa_id = player['sofifa_id']
    profile_image_path = f"Players/{player_sofifa_id}.png"
    if os.path.exists(profile_image_path):
        player_image_sofifa = Image.open(profile_image_path).convert("RGBA")
    else:
        player_image_sofifa = Image.new("RGBA", (1, 1))

    player_image_sofifa = player_image_sofifa.resize((90, 90))
    paste_x = player_image_sofifa.size[0] // 2 + 30
    paste_y = player_image_sofifa.size[1] // 2 - 25
    composed_image.alpha_composite(player_image_sofifa, (paste_x, paste_y))

    best_position_x = flag_x + flag_image.size[0] // 2 - len(player['best_position']) * 5
    best_position_y = flag_y - 20
    best_position_font = ImageFont.truetype("cour.ttf", 18)
    draw.text((best_position_x, best_position_y), player['best_position'], fill=(0, 0, 0), font=best_position_font)

    overall_x = best_position_x - 2
    overall_y = best_position_y - 25
    overall_font = ImageFont.truetype("timesbd.ttf", 25)
    draw.text((overall_x, overall_y), str(player['overall']), fill=(0, 0, 0), font=overall_font)

    pace_x = paste_x + player_image.size[0] // 2 - len(str(player['pace'])) - 125
    pace_y = paste_y + player_image.size[1] - 87
    pace_font = ImageFont.truetype("times.ttf", 11)
    draw.text((pace_x, pace_y), str(player['pace']), fill=(0, 0, 0), font=pace_font)

    shooting_x = pace_x
    shooting_y = pace_y + 15
    shooting_font = ImageFont.truetype("times.ttf", 11)
    draw.text((shooting_x, shooting_y), str(player['shooting']), fill=(0, 0, 0), font=shooting_font)

    passing_x = pace_x
    passing_y = shooting_y + 15
    passing_font = ImageFont.truetype("times.ttf", 11)
    draw.text((passing_x, passing_y), str(player['passing']), fill=(0, 0, 0), font=passing_font)

    dribbling_x = pace_x + 70
    dribbling_y = pace_y
    dribbling_font = ImageFont.truetype("times.ttf", 11)
    draw.text((dribbling_x, dribbling_y), str(player['dribbling_overall']), fill=(0, 0, 0), font=dribbling_font)

    defense_x = dribbling_x
    defense_y = dribbling_y + 15
    defense_font = ImageFont.truetype("times.ttf", 11)
    draw.text((defense_x, defense_y), str(player['defense']), fill=(0, 0, 0), font=defense_font)

    physical_x = dribbling_x
    physical_y = defense_y + 15
    physical_font = ImageFont.truetype("times.ttf", 11)
    draw.text((physical_x, physical_y), str(player['physical']), fill=(0, 0, 0), font=physical_font)

    image_byte_array = io.BytesIO()
    composed_image.save(image_byte_array, format='PNG')
    composed_image_data = image_byte_array.getvalue()

    composed_image_widget = widgets.Image(value=composed_image_data, format='png')

    return composed_image_widget

def update_team_list():
    team_output.clear_output()
    with team_output:
        team_p = widgets.HTML('<h4 style="background-color: #f1c232; font-size: 14px; font-weight: bold; padding: 8px;">' + 'Team Players:'+ '</h4>', layout=widgets.Layout(width='200px'))  # Print heading as an <h4> tag
        display(team_p)
        if len(team_players) > 0:
            # Display budget row
            budget_r = locale.format_string("%d", budget_input.value, grouping=True)
            budget_row = widgets.HBox([
                widgets.HTML('<span style="background-color: #cc0000; font-size: 14px; font-weight: bold; padding: 8px;">' +"Remaining Budget:" + '</span>', layout=widgets.Layout(width='200px')),
                widgets.Label(budget_r, layout=widgets.Layout(width='200px'))
            ])
            display(budget_row)
            
            headers = ['Name', 'Club', 'Nationality', 'Market Value', 'Fifa Card', 'Potential', 'Age', 'Remove from Team']
            # Define header styles
            header_style = "background-color: #f1c232; font-size: 14px; font-weight: bold; padding: 8px;"

            # Create header labels with customized styles
            header_labels = [
                widgets.HTML(value=f'<div style="{header_style}">{header}</div>', layout=widgets.Layout(width='200px'))
                for header in headers
            ]

            # Create empty spacer labels for spacing
            spacer_labels = [
                widgets.Label('', layout=widgets.Layout(width='20px'))
                for _ in range(7)  # Adjust the number of spacers based on the number of columns in the table
            ]

            # Combine header labels and spacer labels
            header_row = widgets.HBox(header_labels + spacer_labels)
            display(header_row)  # Display the header row

            # Display player rows
            for player_index in team_players:
                player = df.loc[player_index]  # Retrieve the player from the DataFrame using the index
                market_value = locale.format_string("%d", player['market_value'], grouping=True)


                composed_image_widget = create_composed_image_widget(df,player_index)
                player_nationality = player['nationality']
                flag_image_path_col = f"Countries/{player_nationality}.png"
                if os.path.exists(flag_image_path_col):
                    flag_image = Image.open(flag_image_path_col).convert("RGBA")
                else:
                    flag_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

                # Resize the flag image
                flag_width, flag_height = 69, 45
                flag_image = flag_image.resize((flag_width, flag_height))

                # Add black borders to the flag image
                border_size = 1
                border_color = (0, 0, 0)  # Black color
                flag_image_with_borders = ImageOps.expand(flag_image, border=border_size, fill=border_color)

                # Convert the flag image to bytes
                byte_array = io.BytesIO()
                flag_image_with_borders.save(byte_array, format='PNG')
                flag_image_data = byte_array.getvalue()

                # Create the flag widget
                flag_widget = widgets.Image(value=flag_image_data, format='png')

                # Create the country name label
                country_label = widgets.Label(player_nationality)

                player_club = player['club_name']
                club_image_path_col = f"Club/{player_club}.png"
                if os.path.exists(club_image_path_col):
                    club_image = Image.open(club_image_path_col).convert("RGBA")
                else:
                    club_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

                # Resize the club image
                club_width, club_height = 200, 180
                club_image = club_image.resize((club_width, club_height))

                # # Add black borders to the club image
                # border_size = 1
                # border_color = (0, 0, 0)  # Black color
                club_image_with_borders = ImageOps.expand(club_image)

                # Convert the club image to bytes
                byte_array = io.BytesIO()
                club_image_with_borders.save(byte_array, format='PNG')
                club_image_data = byte_array.getvalue()

                # Create the club widget
                club_widget = widgets.Image(value=club_image_data, format='png')

                # Create the club name label
                league_name = player['league_name']
                club_label = widgets.HTML('<span style="font-size:16px">' + player_club + '</span>',layout=widgets.Layout(width='200px'))
                league_label = widgets.HTML('<span style="font-size:16px">' +league_name + '</span>',layout=widgets.Layout(width='200px'))


                remove_button = widgets.Button(description='Remove', button_style='danger')
                remove_button.on_click(lambda _, player_index=player_index: remove_from_team(player_index))
                market_value = locale.format_string("%d", player['market_value'], grouping=True)
                row = widgets.HBox([
                    widgets.HTML('<span style="font-size:16px">' + player['name'] + '</span>', layout=widgets.Layout(width='200px')),
                    widgets.VBox([club_widget, club_label,league_label],layout=widgets.Layout(width='100px')),
                    widgets.Label('', layout=widgets.Layout(width='110px')),
                    widgets.VBox([flag_widget, country_label],layout=widgets.Layout(width='100px')),
                    widgets.Label('', layout=widgets.Layout(width='110px')), 
                    widgets.HTML('<span style="font-size:16px">€ ' + market_value + '</span>', layout=widgets.Layout(width='200px')),
                    composed_image_widget,
                    widgets.Label('', layout=widgets.Layout(width='30px')), 
                    widgets.HTML('<span style="font-size:16px">' + str(player['potential']) + '</span>', layout=widgets.Layout(width='200px')),
                    widgets.HTML('<span style="font-size:16px">' + str(player['age']) + '</span>', layout=widgets.Layout(width='200px')),
                    remove_button
                ])
                display(row)

            players_widgets = []  # Declare an empty list to store player widgets

            if len(team_players) == 10:
                # Close or clear the composed image widget if it is currently displayed
                for player_widget in players_widgets:
                    composed_image_widget = player_widget.children[0]
                    composed_image_widget.close()  # Close the widget if it is a IPython display widget
                    # Alternatively, you can clear the widget's output if it is a Jupyter widget
                    composed_image_widget.clear_output()
            elif len(team_players) == 11:
                players_widgets = []
                for player_index in team_players:
                    player_name = df.loc[player_index]['name']
                    positions = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM',
                                'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']

                    player_positions = [(pos, df.loc[player_index][pos]) for pos in positions]
                    sorted_positions = sorted(player_positions, key=lambda x: x[1], reverse=True)  # Sort positions by value

                    position_options = [f"{pos} ({value})" for pos, value in sorted_positions]

                    position_dropdown = widgets.Dropdown(
                        options=position_options,
                        description="Position:",
                        layout=widgets.Layout(width='180px')
                    )
                    composed_image_widget = create_composed_image_widget(df, player_index)
                    player_widgets = widgets.VBox([composed_image_widget, position_dropdown],layout=widgets.Layout(width='200px'))
                    players_widgets.append(player_widgets)

                players_per_line = 9  # Number of players to display per line
                player_lines = [players_widgets[i:i+players_per_line] for i in range(0, len(players_widgets), players_per_line)]
                players_vbox = widgets.VBox([widgets.HBox(line) for line in player_lines])
                players_vbox.layout.margin = "100px 0 100px 0"

                display(players_vbox)

                show_starting_11_button = widgets.Button(description='Show Me My Starting 11', layout=widgets.Layout(width='200px'),button_style='warning')
                display(show_starting_11_button)

                def on_click_show_starting_11(button):
                    selected_positions = []
                    for i, player_widget in enumerate(players_widgets):
                        player_index = team_players[i]
                        position_dropdown = player_widget.children[1]
                        selected_position = position_dropdown.value.split(" (")[0]  # Remove the value in parenthesis
                        selected_positions.append((player_index, selected_position))  # Store player index with selected position
                    show_starting_11(df, selected_positions)

                show_starting_11_button.on_click(on_click_show_starting_11)
            elif len(team_players) == 0:
                print("No players in the team.")
                    
def filter_and_sort_data(_):
    alert.clear_output()
    with filtering:    
        if _ is not None:
            current_page[0] = 0  # Reset current page to the first page when filter button is clicked
        budget = budget_input.value
        league = league_dropdown.value
        club = club_dropdown.value
        continent = continent_dropdown.value
        nationality = nationality_dropdown.value
        position = position_dropdown.value
        overall = overall_input.value
        overall_option = overall_option_button.value
        potential = potential_input.value
        potential_option = potential_option_button.value
        age = age_input.value
        age_option = age_option_button.value
        market_value = market_value_input.value
        market_value_option = market_value_option_button.value
        sort_by = sort_dropdown.value
        sort_order = sort_order_dropdown.value
        page_size = 5  # Number of rows to display per page


        # Filter players by budget, league, continent, nationality, position, overall, potential, age, and market value
        filtered_df = df[(df['market_value'] <= budget) |
                            (budget == 0)]
        if league == 'Top 5 Leagues':
            filtered_df = filtered_df[(filtered_df['league_name'].isin(['[Spain 1st Tier] LaLiga',
                                                                    '[England 1st Tier] Premier League',
                                                                    '[France 1st Tier] Ligue 1',
                                                                    '[Germany 1st Tier] Bundesliga',
                                                                    '[Italy 1st Tier] Serie A']))]
        elif league =='Not in Top 5 League':
            filtered_df = filtered_df[(~filtered_df['league_name'].isin(['[Spain 1st Tier] LaLiga',
                                                                '[England 1st Tier] Premier League',
                                                                '[France 1st Tier] Ligue 1',
                                                                '[Germany 1st Tier] Bundesliga',
                                                                '[Italy 1st Tier] Serie A']))]
        else:
            if league != 'Any':
                filtered_df = filtered_df[(filtered_df['league_name'] == league)]

        if continent != 'Any':
            filtered_df = filtered_df[(filtered_df['continent'] == continent)]

        if nationality != 'Any':
            filtered_df = filtered_df[(filtered_df['nationality'] == nationality)]

        if position != 'Any':
            filtered_df = filtered_df[(filtered_df['best_position'] == position)]

        if club != 'Any':
            filtered_df = filtered_df[(filtered_df['club_name'] == club)]


        # Filter players by overall
        if overall_option == '<=':
            filtered_df = filtered_df[(filtered_df['overall'] <= overall)]
        elif overall_option == '>=':
            filtered_df = filtered_df[(filtered_df['overall'] >= overall)]
        else:
            filtered_df = filtered_df[(filtered_df['overall'] == overall)]

        # Filter players by potential
        if potential_option == '<=':
            filtered_df = filtered_df[(filtered_df['potential'] <= potential)]
        elif potential_option == '>=':
            filtered_df = filtered_df[(filtered_df['potential'] >= potential)]
        else:
            filtered_df = filtered_df[(filtered_df['potential'] == potential)]

        # Filter players by age
        if age_option == '<=':
            filtered_df = filtered_df[(filtered_df['age'] <= age)]
        elif age_option == '>=':
            filtered_df = filtered_df[(filtered_df['age'] >= age)]
        else:
            filtered_df = filtered_df[(filtered_df['age'] == age)]

        # Filter players by market value
        if market_value_option == '<=':
            filtered_df = filtered_df[(filtered_df['market_value'] <= market_value)]
        elif market_value_option == '>=':
            filtered_df = filtered_df[(filtered_df['market_value'] >= market_value)]
        else:
            filtered_df = filtered_df[(filtered_df['market_value'] == market_value)]

        # Filter players based on position and specific attributes
        if sort_by in ['Pace', 'Shooting', 'Passing', 'Dribling', 'Defending','Physical']:
            filtered_df = filtered_df[filtered_df['best_position'] != 'GK']
        elif sort_by in ['Diving', 'Handling', 'Kicking', 'Reflexes', 'Speed', 'Positioning']:
            filtered_df = filtered_df[filtered_df['best_position'] == 'GK']

        # Sort the filtered results
        if sort_by == 'Overall':
            filtered_df = filtered_df.sort_values('overall', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Potential':
            filtered_df = filtered_df.sort_values('potential', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Age':
            filtered_df = filtered_df.sort_values('age', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Market Value':
            filtered_df = filtered_df.sort_values('market_value', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Pace':
            filtered_df = filtered_df.sort_values('pace', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Shooting':
            filtered_df = filtered_df.sort_values('shooting', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Passing':
            filtered_df = filtered_df.sort_values('passing', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Dribling':
            filtered_df = filtered_df.sort_values('dribbling_overall', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Defending':
            filtered_df = filtered_df.sort_values('defense', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Physical':
            filtered_df = filtered_df.sort_values('physical', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Diving':
            filtered_df = filtered_df.sort_values('pace', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Handling':
            filtered_df = filtered_df.sort_values('shooting', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Kicking':
            filtered_df = filtered_df.sort_values('passing', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Reflexes':
            filtered_df = filtered_df.sort_values('dribbling_overall', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Speed':
            filtered_df = filtered_df.sort_values('defense', ascending=(sort_order == 'Ascending'))
        elif sort_by == 'Positioning':
            filtered_df = filtered_df.sort_values('physical', ascending=(sort_order == 'Ascending'))

        # Calculate the total number of pages
        total_pages = (filtered_df.shape[0] - 1) // page_size + 1

        # Display the current page of results
        display_page = filtered_df.iloc[current_page[0] * page_size:(current_page[0] + 1) * page_size]


        headers = ['Name', 'Club', 'Nationality', 'Market Value', 'Fifa Card', 'Potential', 'Age', 'Add to Team']
        # Define header styles
        header_style = "background-color: #cfe2f3; font-size: 14px; font-weight: bold; padding: 8px;"

        # Create header labels with customized styles
        header_labels = [
            widgets.HTML(value=f'<div style="{header_style}">{header}</div>', layout=widgets.Layout(width='200px'))
            for header in headers
        ]

        # Create empty spacer labels for spacing
        spacer_labels = [
            widgets.Label('', layout=widgets.Layout(width='20px'))
            for _ in range(7)  # Adjust the number of spacers based on the number of columns in the table
        ]

        # Combine header labels and spacer labels
        header_row = widgets.HBox(header_labels + spacer_labels)

        rows = []
        table = []
        for index, player in display_page.iterrows():
            add_button = add_to_team(player)
            market_value = locale.format_string("%d", player['market_value'], grouping=True)

            paste_x = 0
            paste_y = 0

                    # Load player image based on overall rating and position
            if player['overall'] >= 75:
                if player['best_position'] == 'GK':
                    player_image = Image.open('Fifa cards/Gold GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Gold.png').convert("RGBA")
            elif player['overall'] >= 65:
                if player['best_position'] == 'GK':
                    player_image = Image.open('Fifa cards/Silver GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Silver.png').convert("RGBA")
            else:
                if player['best_position'] == 'GK':
                    player_image = Image.open('Fifa cards/Bronze GK.png').convert("RGBA")
                else:
                    player_image = Image.open('Fifa cards/Bronze.png').convert("RGBA")

            # Resize the player image
            player_image = player_image.resize((200, 200))

            # Create a transparent background image
            transparent_background = Image.new("RGBA", player_image.size, (0, 0, 0, 0))

            # Paste the player image onto the transparent background
            composed_image = Image.alpha_composite(transparent_background, player_image)

            if len(player['name'].split()) > 1:
                first_name, *rest_of_names = player['name'].split()
                displayed_name = f"{first_name[0]}. {' '.join(rest_of_names)}"

            else:
                displayed_name = player['name']


            # Add the name below the player image
            name_x = paste_x + player_image.size[0] // 2 - len(displayed_name) * 4
            name_y = paste_y + player_image.size[1] - 90
            name_font = ImageFont.truetype("times.ttf", 18)
            draw = ImageDraw.Draw(composed_image)
            draw.text((name_x, name_y), displayed_name, fill=(0, 0, 0), font=name_font)

            # Load the nationality flag image
            player_nationality = player['nationality']
            flag_image_path = f"Countries/{player_nationality}.png"
            if os.path.exists(flag_image_path):
                flag_image = Image.open(flag_image_path).convert("RGBA")
            else:
                flag_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

            # Resize the flag image
            flag_image = flag_image.resize((23, 15))

            # Calculate the coordinates to paste the flag image
            flag_x = paste_x - flag_image.size[0] + 68
            flag_y = paste_y + (player_image.size[1] - flag_image.size[1]) // 2 - 26

            # Paste the flag image on the composed image
            composed_image.alpha_composite(flag_image, (flag_x, flag_y))

            # Load the team logo image
            player_club = player['club_name']
            team_logo_path = f"Club/{player_club}.png"
            if os.path.exists(team_logo_path):
                team_logo = Image.open(team_logo_path).convert("RGBA")
            else:
                team_logo = Image.new("RGBA", (1, 1))  # Use a blank image if team logo doesn't exist

            # Resize the team logo image
            team_logo = team_logo.resize((22, 22))

            # Calculate the coordinates to paste the team logo image
            logo_x = flag_x
            logo_y = flag_y + flag_image.size[1] + 7

            # Paste the team logo image on the composed image
            composed_image.alpha_composite(team_logo, (logo_x, logo_y))

            # Load the Sofifa profile image
            player_sofifa_id = player['sofifa_id']
            profile_image_path = f"Players/{player_sofifa_id}.png"
            if os.path.exists(profile_image_path):
                player_image_sofifa = Image.open(profile_image_path).convert("RGBA")
            else:
                player_image_sofifa = Image.new("RGBA", (1, 1))  # Use a blank image if profile image doesn't exist

            # Resize the Sofifa image
            player_image_sofifa = player_image_sofifa.resize((90, 90))
            paste_x =  player_image_sofifa.size[0] // 2 +30
            paste_y =  player_image_sofifa.size[1] // 2 -25
            # Paste the Sofifa image on top of the player image
            composed_image.alpha_composite(player_image_sofifa, (paste_x, paste_y))


            # Add the best position above the flag
            best_position_x = flag_x + flag_image.size[0] // 2 - len(player['best_position']) * 5
            best_position_y = flag_y - 20
            best_position_font = ImageFont.truetype("cour.ttf", 18)
            draw.text((best_position_x, best_position_y), player['best_position'], fill=(0, 0, 0), font=best_position_font)

            # Add the overall rating above the best position
            overall_x = best_position_x -2
            overall_y = best_position_y - 25
            overall_font = ImageFont.truetype("timesbd.ttf", 25)
            draw.text((overall_x, overall_y), str(player['overall']), fill=(0, 0, 0), font=overall_font)

                    # Add the pace attribute below the name
            pace_x = paste_x + player_image.size[0] // 2 - len(str(player['pace']))-125
            pace_y = paste_y + player_image.size[1]-87
            pace_font = ImageFont.truetype("times.ttf", 11)
            draw.text((pace_x, pace_y), str(player['pace']), fill=(0, 0, 0), font=pace_font)

            # Add the shooting attribute below pace
            shooting_x = pace_x
            shooting_y = pace_y + 15
            shooting_font = ImageFont.truetype("times.ttf", 11)
            draw.text((shooting_x, shooting_y), str(player['shooting']), fill=(0, 0, 0), font=shooting_font)

            # Add the passing attribute below shooting
            passing_x = pace_x
            passing_y = shooting_y + 15
            passing_font = ImageFont.truetype("times.ttf", 11)
            draw.text((passing_x, passing_y), str(player['passing']), fill=(0, 0, 0), font=passing_font)

            # Add the dribbling overall attribute to the right of pace
            dribbling_x = pace_x + 70
            dribbling_y = pace_y
            dribbling_font = ImageFont.truetype("times.ttf", 11)
            draw.text((dribbling_x, dribbling_y), str(player['dribbling_overall']), fill=(0, 0, 0), font=dribbling_font)

            # Add the defense attribute below dribbling overall
            defense_x = dribbling_x
            defense_y = dribbling_y + 15
            defense_font = ImageFont.truetype("times.ttf", 11)
            draw.text((defense_x, defense_y), str(player['defense']), fill=(0, 0, 0), font=defense_font)

            # Add the physical attribute below defense
            physical_x = dribbling_x
            physical_y = defense_y + 15
            physical_font = ImageFont.truetype("times.ttf", 11)
            draw.text((physical_x, physical_y), str(player['physical']), fill=(0, 0, 0), font=physical_font)

            # Convert the composed image to bytes
            image_byte_array = io.BytesIO()
            composed_image.save(image_byte_array, format='PNG')
            composed_image_data = image_byte_array.getvalue()
            composed_image_widget = widgets.Image(value=composed_image_data, format='png')

            flag_image_path_col = f"Countries/{player_nationality}.png"
            if os.path.exists(flag_image_path_col):
                flag_image = Image.open(flag_image_path_col).convert("RGBA")
            else:
                flag_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

            # Resize the flag image
            flag_width, flag_height = 69, 45
            flag_image = flag_image.resize((flag_width, flag_height))

            # Add black borders to the flag image
            border_size = 1
            border_color = (0, 0, 0)  # Black color
            flag_image_with_borders = ImageOps.expand(flag_image, border=border_size, fill=border_color)

            # Convert the flag image to bytes
            byte_array = io.BytesIO()
            flag_image_with_borders.save(byte_array, format='PNG')
            flag_image_data = byte_array.getvalue()

            # Create the flag widget
            flag_widget = widgets.Image(value=flag_image_data, format='png')

            # Create the country name label
            country_label = widgets.Label(player_nationality)


            club_image_path_col = f"Club/{player_club}.png"
            if os.path.exists(club_image_path_col):
                club_image = Image.open(club_image_path_col).convert("RGBA")
            else:
                club_image = Image.new("RGBA", (1, 1))  # Use a blank image if flag image doesn't exist

            # Resize the club image
            club_width, club_height = 200, 180
            club_image = club_image.resize((club_width, club_height))

            # # Add black borders to the club image
            # border_size = 1
            # border_color = (0, 0, 0)  # Black color
            club_image_with_borders = ImageOps.expand(club_image)

            # Convert the club image to bytes
            byte_array = io.BytesIO()
            club_image_with_borders.save(byte_array, format='PNG')
            club_image_data = byte_array.getvalue()

            # Create the club widget
            club_widget = widgets.Image(value=club_image_data, format='png')

            # Create the club name label
            league_name = player['league_name']
            club_label = widgets.HTML('<span style="font-size:16px">' + player_club + '</span>',layout=widgets.Layout(width='200px'))
            league_label = widgets.HTML('<span style="font-size:16px">' +league_name + '</span>',layout=widgets.Layout(width='200px'))

            
            # Create the row with all the widgets
            row = widgets.HBox([
                widgets.HTML('<span style="font-size:16px">' + player['name'] + '</span>', layout=widgets.Layout(width='200px')),
                widgets.VBox([club_widget, club_label,league_label],layout=widgets.Layout(width='100px')),
                widgets.Label('', layout=widgets.Layout(width='110px')),
                widgets.VBox([flag_widget, country_label],layout=widgets.Layout(width='100px')),
                widgets.Label('', layout=widgets.Layout(width='110px')), 
                widgets.HTML('<span style="font-size:16px">€ ' + market_value + '</span>', layout=widgets.Layout(width='200px')),
                composed_image_widget,
                widgets.Label('', layout=widgets.Layout(width='30px')), 
                widgets.HTML('<span style="font-size:16px">' + str(player['potential']) + '</span>', layout=widgets.Layout(width='200px')),
                widgets.HTML('<span style="font-size:16px">' + str(player['age']) + '</span>', layout=widgets.Layout(width='200px')),
                add_button
            ])

            row.layout.align_self = 'auto'
            rows.append(row)
            table = widgets.VBox([header_row, *rows,alert])
                
        # Pagination buttons
        prev_button = widgets.Button(description='Previous',button_style='info')
        next_button = widgets.Button(description='Next',button_style='info')

        def show_previous(_):
            alert.clear_output()
            if current_page[0] > 0:
                current_page[0] -= 1
            filter_and_sort_data(None)

        def show_next(_):
            alert.clear_output()
            if current_page[0] < total_pages - 1:
                current_page[0] += 1
            filter_and_sort_data(None)

        prev_button.on_click(show_previous)
        next_button.on_click(show_next)

        # Disable "Previous" button if on the first page
        prev_button.disabled = current_page[0] == 0

        # Disable "Next" button if on the last page
        next_button.disabled = current_page[0] >= total_pages - 1

        # Update output
        with output:
            output.clear_output()
            if not table:
                message_label = widgets.Label("No players match your criteria.")
                display(message_label)
            else:
                display(table)
                display(widgets.HBox([prev_button, next_button]))

# Step 5: Add event handlers for interactive widgets
continent_dropdown.observe(update_nationalities, 'value')
league_dropdown.observe(update_clubs, 'value')  # Register the update_clubs function as an event handler for the league dropdown

filter_button.on_click(filter_and_sort_data)

# Step 6: Display the interactive widgets and output
output_box = widgets.Output()
display(widgets.VBox([budget_input, display_button,output_box]))
output = widgets.Output()
# Add pagination buttons
team_output = widgets.Output()

HTML(value='<h5 style="background-color: #77ffe4; font-size: 14px; font-weight: bold; padding: 8px;">Select yo…

Output()

Output()

Output()